In [20]:
import numpy as np

#-----------------------------------------------
# Initialization
#-----------------------------------------------

MAP='CAMA'
loop_max = 8
num = 23 # 23: Jakarta

#-----------------------------------------------
# load true data (UN city list)
#-----------------------------------------------

pop_list = []
name_list = []
inf_path = '/home/kajiyama/H08/H08_20230612/map/dat/cty_lst_/city_list03.txt'
for l in open(inf_path).readlines():
    data = l[:-1].split('	')
    pop_list.append(int(data[4]))
    name_list.append(data[5])
    
#-----------------------------------------------
# load population and ared data
#-----------------------------------------------

# population data(GWP4 2000)
pop = np.fromfile('/home/kajiyama/H08/H08_20230612/map/dat/pop_tot_/C05_a___20000000.gl5', 'float32').reshape(2160, 4320)
area = np.fromfile(f'/home/kajiyama/H08/H08_20230612/map/dat/lnd_ara_/lndara.{MAP}.gl5', 'float32').reshape(2160, 4320)

#-----------------------------------------------
# country id
#-----------------------------------------------
num = 23 # 23: Jakarta

# threshold
tt_1 = 1000
tt_2 = 2000
tt_3 = 3000
tt_list = [1000,2000,3000]

# city center
location = np.fromfile(f'/home/kajiyama/H08/H08_20230612/map/dat/cty_cnt_/city_{num:08d}.gl5','float32').reshape(2160,4320)
pop_city = pop_list[num-1]*1000
city_name = name_list[num-1]
x = np.where(location==1)[0]
y = np.where(location==1)[1]
x = x[0]
y = y[0]

#-------------------ROUND 1----------------------
A = area[x,y]/1000./1000.
thre_1 = tt_1*A
thre_2 = tt_2*A
thre_3 = tt_3*A

# 保存するマスクファイル
mask_1 = np.zeros((2160,4320),'float32')
mask_1[x,y] = 1
mask_2 = np.zeros((2160,4320),'float32')
mask_2[x,y] = 1
mask_3 = np.zeros((2160,4320),'float32')
mask_3[x,y] = 1

# 探索用ファイル
search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1

# サーチするグリッド数
for a in range(x-1, x+2):
    for b in range(y-1, y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)

# 中心グリッドからの半径
ran = 1

# maskのコピーファイル
f_mask_1 = np.zeros((2160,4320),'float32')
f_mask_2 = np.zeros((2160,4320),'float32')
f_mask_3 = np.zeros((2160,4320),'float32')

#-----------------------------------------------
loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_1:
                    if mask_1[a-1,b-1]==1 or mask_1[a,b-1]==1 or mask_1[a+1,b-1]==1 or mask_1[a-1,b]==1 or mask_1[a+1,b]==1 or mask_1[a-1,b+1]==1 or mask_1[a,b+1]==1 or mask_1[a+1,b+1]==1:
                        mask_1[a,b] = 1
                        print(f"thre_1: a{a}, b {b}")
                elif pop[a,b]>thre_2:
                    if mask_2[a-1,b-1]==1 or mask_2[a,b-1]==1 or mask_2[a+1,b-1]==1 or mask_2[a-1,b]==1 or mask_2[a+1,b]==1 or mask_2[a-1,b+1]==1 or mask_2[a,b+1]==1 or mask_2[a+1,b+1]==1:
                        mask_2[a,b] = 1
                        print(f"thre_2: a{a}, b {b}")
                elif pop[a,b]>thre_3:
                    if mask_3[a-1,b-1]==1 or mask_3[a,b-1]==1 or mask_3[a+1,b-1]==1 or mask_3[a-1,b]==1 or mask_3[a+1,b]==1 or mask_3[a-1,b+1]==1 or mask_3[a,b+1]==1 or mask_3[a+1,b+1]==1:
                        mask_3[a,b] = 1
                        print(f"thre_3: a{a}, b {b}")

    # 探索半径の更新
    ran = ran + 1
    
    # 既に探索した部分をさ駆除
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    # 依然とのマスクの増加分を記録
    judgement_1 = np.sum(mask_1-f_mask_1)
    judgement_2 = np.sum(mask_2-f_mask_2)
    judgement_3 = np.sum(mask_3-f_mask_3)
    
    # マスクのコピーを更新
    f_mask_1 = np.copy(mask_1)
    f_mask_2 = np.copy(mask_2)
    f_mask_3 = np.copy(mask_3)

    # 増加分が最大の閾値を記録
    JG = [judgement_1,judgement_2,judgement_3]
    judgement = max(JG)
    
    # evaluate coverage
    mp_1 = mask_1*pop
    pop_mask_1 = np.sum(mp_1)
    mp_2 = mask_2*pop
    pop_mask_2 = np.sum(mp_2)
    mp_3 = mask_3*pop
    pop_mask_3 = np.sum(mp_3)
    pop_mask_list = [pop_mask_1,pop_mask_2,pop_mask_3]
    coverage = [float(M/pop_city) for M in pop_mask_list]
    evl = [abs(C-1.0) for C in coverage]
    min_evl = min(evl)
    best_index = evl.index(min_evl)
    best_coverage = coverage[best_index]
    if best_coverage >= 1.0:
        loop = loop_max
    else:
        # ループを更新
        loop += 1
#-----------------------------------------------

print(f"pop_mask_list: {pop_mask_list}")
print(f"coverage: {coverage}")
print(f"evl: {evl}")

#-------------------ROUND 2----------------------
if best_index==0:
    tt_4 = 500
    tt_best = 1000
    tt_5 = 1500
    pop_mask_best = pop_mask_list[0]
    best_mask = mask_1
elif best_index==1:
    tt_4 = 1500
    tt_best = 2000
    tt_5 = 2500
    pop_mask_best = pop_mask_list[1]
    best_mask = mask_2
elif best_index==2:
    tt_4 = 2500
    tt_best = 3000
    tt_5 = 3500
    pop_mask_best = pop_mask_list[2]
    best_mask = mask_3

tt_list = [tt_4,tt_best,tt_5]
thre_4 = tt_4*A
thre_5 = tt_5*A
thre_list = [tt_4*A,tt_best*A,tt_5*A]

mask_4 = np.zeros((2160,4320),'float32')
mask_4[x,y] = 1
mask_5 = np.zeros((2160,4320),'float32')
mask_5[x,y] = 1

search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1
for a in range(x-1,x+2):
    for b in range(y-1,y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)
ran = 1

f_mask_4 = np.zeros((2160,4320),'float32')
f_mask_5 = np.zeros((2160,4320),'float32')

#-----------------------------------------------
loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_4:
                    if mask_4[a-1,b-1]==1 or mask_4[a,b-1]==1 or mask_4[a+1,b-1]==1 or mask_4[a-1,b]==1 or mask_4[a+1,b]==1 or mask_4[a-1,b+1]==1 or mask_4[a,b+1]==1 or mask_4[a+1,b+1]==1:
                        mask_4[a,b] = 1
                        print(f"thre_4: a{a}, b {b}")
                elif pop[a,b]>thre_5:
                    if mask_5[a-1,b-1]==1 or mask_5[a,b-1]==1 or mask_5[a+1,b-1]==1 or mask_5[a-1,b]==1 or mask_5[a+1,b]==1 or mask_5[a-1,b+1]==1 or mask_5[a,b+1]==1 or mask_5[a+1,b+1]==1:
                        mask_5[a,b] = 1
                        print(f"thre_4: a{a}, b {b}")

    ran = ran + 1
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    judgement_4 = np.sum(mask_4-f_mask_4)
    f_mask_4 = np.copy(mask_4)
    judgement_5 = np.sum(mask_5-f_mask_5)
    f_mask_5 = np.copy(mask_5)

    JG = [judgement_4,judgement_5]
    judgement = max(JG)
    
    # evaluate coverage
    mp_4 = mask_4*pop
    pop_mask_4 = np.sum(mp_4)
    mp_5 = mask_5*pop
    pop_mask_5 = np.sum(mp_5)
    pop_mask_list_2 = [pop_mask_4,pop_mask_best,pop_mask_5]
    print(f"pop_mask_list_2: {pop_mask_list_2}")
    coverage_2 = [float(M/pop_city) for M in pop_mask_list_2]
    print(f"coverage_2: {coverage_2}")
    evl_2 = [abs(C-1.0) for C in coverage_2]
    print(f"evl_2: {evl_2}")
    min_evl_2 = min(evl_2)
    best_index_2 = evl_2.index(min_evl_2)
    best_coverage_2 = coverage_2[best_index_2]
    
    if best_coverage_2 >= 1.0:
        loop = loop_max
    else:
        loop += 1
#-----------------------------------------------

pop_best_2 = pop_mask_list_2[best_index_2]
tt_best_2 = tt_list[best_index_2]
thre_best_2 = thre_list[best_index_2]

#-------------------ROUND 3----------------------
if best_index_2==0:
    best_mask_2 = mask_4
elif best_index_2==1:
    best_mask_2 = best_mask
elif best_index_2==2:
    best_mask_2 = mask_5

tt_6 = tt_best_2-250
tt_7 = tt_best_2+250

tt_list = [tt_6,tt_best_2,tt_7]
thre_6 = tt_6*A
thre_7 = tt_7*A
thre_list = [thre_6,thre_best_2,thre_7]

mask_6 = np.zeros((2160,4320),'float32')
mask_6[x,y] = 1
mask_7 = np.zeros((2160,4320),'float32')
mask_7[x,y] = 1

search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1
for a in range(x-1,x+2):
    for b in range(y-1,y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)
ran = 1

f_mask_6 = np.zeros((2160,4320),'float32')
f_mask_7 = np.zeros((2160,4320),'float32')

loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_6:
                    if mask_6[a-1,b-1]==1 or mask_6[a,b-1]==1 or mask_6[a+1,b-1]==1 or mask_6[a-1,b]==1 or mask_6[a+1,b]==1 or mask_6[a-1,b+1]==1 or mask_6[a,b+1]==1 or mask_6[a+1,b+1]==1:
                        mask_6[a,b] = 1
                        print(f"thre_6: a{a}, b {b}")
                elif pop[a,b]>thre_7:
                    if mask_7[a-1,b-1]==1 or mask_7[a,b-1]==1 or mask_7[a+1,b-1]==1 or mask_7[a-1,b]==1 or mask_7[a+1,b]==1 or mask_7[a-1,b+1]==1 or mask_7[a,b+1]==1 or mask_7[a+1,b+1]==1:
                        mask_7[a,b] = 1
                        print(f"thre_7: a{a}, b {b}")

    ran = ran + 1
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    judgement_6 = np.sum(mask_6-f_mask_6)
    f_mask_6 = np.copy(mask_6)
    judgement_7 = np.sum(mask_7-f_mask_7)
    f_mask_7 = np.copy(mask_7)
    JG = [judgement_6,judgement_7]
    judgement = max(JG)

    # evaluate covearge
    mp_6 = mask_6*pop
    pop_mask_6 = np.sum(mp_6)
    mp_7 = mask_7*pop
    pop_mask_7 = np.sum(mp_7)
    pop_mask_list_3 = [pop_mask_6,pop_best_2,pop_mask_7]
    print(f"pop_mask_list_3: {pop_mask_list_3}")
    coverage_3 = [float(M/pop_city) for M in pop_mask_list_3]
    print(f"coverage_3: {coverage_3}")
    evl_3 = [abs(C-1.0) for C in coverage_3]
    print(f"evl_3: {evl_3}")
    min_evl_3 = min(evl_3)
    best_index_3 = evl_3.index(min_evl_3)
    best_coverage_3 = coverage_3[best_index_3]
    if best_coverage_3 >= 1.0:
        loop = loop_max
    else:
        loop += 1
#-----------------------------------------------

pop_best_3 = pop_mask_list_3[best_index_3]
tt_best_3 = tt_list[best_index_3]
thre_best_3 = thre_list[best_index_3]

#-----------------------------------------------

if best_index_3==0:
    best_mask_3 = mask_6
elif best_index_3==1:
    best_mask_3 = best_mask_2
elif best_index_3==2:
    best_mask_3 = mask_7

grid_num = np.sum(best_mask_3)
city_A = best_mask_3*A
city_area = np.sum(city_A)

#------------------------------------------------
if tt_best_3 != 3750 and tt_best_3 != 250 and 0.8<=best_coverage_3<=1.2:
    # f"citynum {num}\n" \ #city number
    # f"threshold_1 {tt_best_3}\n" \ # population density threshold 
    # f"threshold_2 {thre_best_3}\n" \ # center population threshold
    # f"pop_mask {pop_best_3}\n" \ # estimated population
    # f"pop_city {pop_city}\n" \ # UN city list population
    # f"coverage {best_coverage_3}\n" \ # coverage rate
    # f"OK grid_num {grid_num}\n" \ # the number of city mask grids
    # f"city_area {city_area}\n" \ # city area
    # f"{city_name}\n") # city name
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"OK grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 != 3750 and tt_best_3 != 250:
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"NG grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 == 3750:
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"high grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 == 250:
    if 0.8<=best_coverage_3<=1.2:
        print(f"citynum {num}\n" \
              f"threshold_1 {tt_best_3}\n" \
              f"threshold_2 {thre_best_3}\n" \
              f"pop_mask {pop_best_3}\n" \
              f"pop_city {pop_city}\n" \
              f"coverage {best_coverage_3}\n" \
              f"low_OK grid_num {grid_num}\n" \
              f"city_area {city_area}\n" \
              f"{city_name}\n")
    else:
        print(f"citynum {num}\n" \
              f"threshold_1 {tt_best_3}\n" \
              f"threshold_2 {thre_best_3}\n" \
              f"pop_mask {pop_best_3}\n" \
              f"pop_city {pop_city}\n" \
              f"coverage {best_coverage_3}\n" \
              f"low_NG grid_num {grid_num}\n" \
              f"city_area {city_area}\n" \
              f"{city_name}\n")

name = f'../../map/dat/cty_msk_/txt/city_{num}.txt'

################## ran = 1 ##################
thre_1: a1153, b 3441
thre_1: a1153, b 3442
thre_1: a1153, b 3443
thre_1: a1154, b 3441
thre_1: a1154, b 3442
thre_1: a1154, b 3443
thre_1: a1155, b 3441
thre_1: a1155, b 3442
thre_1: a1155, b 3443
pop_mask_list: [8673663.0, 1649652.5, 1649652.5]
coverage: [1.0338096543504172, 0.19662127532777116, 0.19662127532777116]
evl: [0.03380965435041716, 0.8033787246722288, 0.8033787246722288]
################## ran = 1 ##################
thre_4: a1153, b 3441
thre_4: a1153, b 3442
thre_4: a1153, b 3443
thre_4: a1154, b 3441
thre_4: a1154, b 3442
thre_4: a1154, b 3443
thre_4: a1155, b 3441
thre_4: a1155, b 3442
thre_4: a1155, b 3443
pop_mask_list_2: [8673663.0, 8673663.0, 1649652.5]
coverage_2: [1.0338096543504172, 1.0338096543504172, 0.19662127532777116]
evl_2: [0.03380965435041716, 0.03380965435041716, 0.8033787246722288]
################## ran = 1 ##################
thre_6: a1153, b 3441
thre_6: a1153, b 3442
thre_6: a1153, b 3443
thre_6: a1154, b

In [ ]:
import numpy as np

MAP='CAMA'
loop_max = 3

# true data(UN city list)
pop_list = []
name_list = []
inf_path = '/home/kajiyama/H08/H08_20230612/map/dat/cty_lst_/city_list03.txt'
for l in open(inf_path).readlines():
    data = l[:-1].split('	')
    pop_list.append(int(data[4]))
    name_list.append(data[5])

# population data(GWP4 2000)
pop = np.fromfile('/home/kajiyama/H08/H08_20230612/map/dat/pop_tot_/C05_a___20000000.gl5', 'float32').reshape(2160, 4320)
area = np.fromfile(f'/home/kajiyama/H08/H08_20230612/map/dat/lnd_ara_/lndara.{MAP}.gl5', 'float32').reshape(2160, 4320)
pop_d = (pop/area/1000./1000.)

#-----------------------------------------------
# country id
num = 23 # 23: Jakarta

# threshold
tt_1 = 1000
tt_2 = 2000
tt_3 = 3000
tt_list = [1000,2000,3000]

# city center
location = np.fromfile(f'/home/kajiyama/H08/H08_20230612/map/dat/cty_cnt_/city_{num:08d}.gl5','float32').reshape(2160,4320)
pop_city = pop_list[num-1]*1000
city_name = name_list[num-1]
x = np.where(location==1)[0]
y = np.where(location==1)[1]
x = x[0]
y = y[0]

#-------------------ROUND 1----------------------
A = area[x,y]/1000./1000.
thre_1 = tt_1*A
thre_2 = tt_2*A
thre_3 = tt_3*A

# 保存するマスクファイル
mask_1 = np.zeros((2160,4320),'float32')
mask_1[x,y] = 1
mask_2 = np.zeros((2160,4320),'float32')
mask_2[x,y] = 1
mask_3 = np.zeros((2160,4320),'float32')
mask_3[x,y] = 1

# 探索用ファイル
search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1

# サーチするグリッド数
for a in range(x-1, x+2):
    for b in range(y-1, y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)

# 中心グリッドからの半径
ran = 1

# maskのコピーファイル
f_mask_1 = np.zeros((2160,4320),'float32')
f_mask_2 = np.zeros((2160,4320),'float32')
f_mask_3 = np.zeros((2160,4320),'float32')

#-----------------------------------------------
loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_1:
                    if mask_1[a-1,b-1]==1 or mask_1[a,b-1]==1 or mask_1[a+1,b-1]==1 or mask_1[a-1,b]==1 or mask_1[a+1,b]==1 or mask_1[a-1,b+1]==1 or mask_1[a,b+1]==1 or mask_1[a+1,b+1]==1:
                        mask_1[a,b] = 1
                        print(f"thre_1: a{a}, b {b}")
                elif pop[a,b]>thre_2:
                    if mask_2[a-1,b-1]==1 or mask_2[a,b-1]==1 or mask_2[a+1,b-1]==1 or mask_2[a-1,b]==1 or mask_2[a+1,b]==1 or mask_2[a-1,b+1]==1 or mask_2[a,b+1]==1 or mask_2[a+1,b+1]==1:
                        mask_2[a,b] = 1
                        print(f"thre_2: a{a}, b {b}")
                elif pop[a,b]>thre_3:
                    if mask_3[a-1,b-1]==1 or mask_3[a,b-1]==1 or mask_3[a+1,b-1]==1 or mask_3[a-1,b]==1 or mask_3[a+1,b]==1 or mask_3[a-1,b+1]==1 or mask_3[a,b+1]==1 or mask_3[a+1,b+1]==1:
                        mask_3[a,b] = 1
                        print(f"thre_3: a{a}, b {b}")

    # 探索半径の更新
    ran = ran + 1
    
    # 既に探索した部分をさ駆除
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    # 依然とのマスクの増加分を記録
    judgement_1 = np.sum(mask_1-f_mask_1)
    judgement_2 = np.sum(mask_2-f_mask_2)
    judgement_3 = np.sum(mask_3-f_mask_3)
    
    # マスクのコピーを更新
    f_mask_1 = np.copy(mask_1)
    f_mask_2 = np.copy(mask_2)
    f_mask_3 = np.copy(mask_3)

    # 増加分が最大の閾値を記録
    JG = [judgement_1,judgement_2,judgement_3]
    judgement = max(JG)
    
    loop += 1
    
#-----------------------------------------------

mp_1 = mask_1*pop
pop_mask_1 = np.sum(mp_1)
mp_2 = mask_2*pop
pop_mask_2 = np.sum(mp_2)
mp_3 = mask_3*pop
pop_mask_3 = np.sum(mp_3)

pop_mask_list = [pop_mask_1,pop_mask_2,pop_mask_3]
print(f"pop_mask_list: {pop_mask_list}")
coverage = [float(M/pop_city) for M in pop_mask_list]
print(f"coverage: {coverage}")
evl = [abs(C-1.0) for C in coverage]
print(f"evl: {evl}")
min_evl = min(evl)
best_index = evl.index(min_evl)
best_coverage = coverage[best_index]

#-------------------ROUND 2----------------------
if best_index==0:
    tt_4 = 500
    tt_best = 1000
    tt_5 = 1500
    pop_mask_best = pop_mask_list[0]
    best_mask = mask_1
elif best_index==1:
    tt_4 = 1500
    tt_best = 2000
    tt_5 = 2500
    pop_mask_best = pop_mask_list[1]
    best_mask = mask_2
elif best_index==2:
    tt_4 = 2500
    tt_best = 3000
    tt_5 = 3500
    pop_mask_best = pop_mask_list[2]
    best_mask = mask_3

tt_list = [tt_4,tt_best,tt_5]
thre_4 = tt_4*A
thre_5 = tt_5*A
thre_list = [tt_4*A,tt_best*A,tt_5*A]

mask_4 = np.zeros((2160,4320),'float32')
mask_4[x,y] = 1
mask_5 = np.zeros((2160,4320),'float32')
mask_5[x,y] = 1

search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1
for a in range(x-1,x+2):
    for b in range(y-1,y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)
ran = 1

f_mask_4 = np.zeros((2160,4320),'float32')
f_mask_5 = np.zeros((2160,4320),'float32')

#-----------------------------------------------
loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_4:
                    if mask_4[a-1,b-1]==1 or mask_4[a,b-1]==1 or mask_4[a+1,b-1]==1 or mask_4[a-1,b]==1 or mask_4[a+1,b]==1 or mask_4[a-1,b+1]==1 or mask_4[a,b+1]==1 or mask_4[a+1,b+1]==1:
                        mask_4[a,b] = 1
                        print(f"thre_4: a{a}, b {b}")
                elif pop[a,b]>thre_5:
                    if mask_5[a-1,b-1]==1 or mask_5[a,b-1]==1 or mask_5[a+1,b-1]==1 or mask_5[a-1,b]==1 or mask_5[a+1,b]==1 or mask_5[a-1,b+1]==1 or mask_5[a,b+1]==1 or mask_5[a+1,b+1]==1:
                        mask_5[a,b] = 1
                        print(f"thre_4: a{a}, b {b}")

    ran = ran + 1
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    judgement_4 = np.sum(mask_4-f_mask_4)
    f_mask_4 = np.copy(mask_4)
    judgement_5 = np.sum(mask_5-f_mask_5)
    f_mask_5 = np.copy(mask_5)

    JG = [judgement_4,judgement_5]
    judgement = max(JG)
    
    loop += 1
#-----------------------------------------------

mp_4 = mask_4*pop
pop_mask_4 = np.sum(mp_4)
mp_5 = mask_5*pop
pop_mask_5 = np.sum(mp_5)

pop_mask_list_2 = [pop_mask_4,pop_mask_best,pop_mask_5]
print(f"pop_mask_list_2: {pop_mask_list_2}")
coverage_2 = [float(M/pop_city) for M in pop_mask_list_2]
print(f"coverage_2: {coverage_2}")
evl_2 = [abs(C-1.0) for C in coverage_2]
print(f"evl_2: {evl_2}")
min_evl_2 = min(evl_2)
best_index_2 = evl_2.index(min_evl_2)
best_coverage_2 = coverage_2[best_index_2]
pop_best_2 = pop_mask_list_2[best_index_2]
tt_best_2 = tt_list[best_index_2]
thre_best_2 = thre_list[best_index_2]

#-------------------ROUND 3----------------------
if best_index_2==0:
    best_mask_2 = mask_4
elif best_index_2==1:
    best_mask_2 = best_mask
elif best_index_2==2:
    best_mask_2 = mask_5

tt_6 = tt_best_2-250
tt_7 = tt_best_2+250

tt_list = [tt_6,tt_best_2,tt_7]
thre_6 = tt_6*A
thre_7 = tt_7*A
thre_list = [thre_6,thre_best_2,thre_7]

mask_6 = np.zeros((2160,4320),'float32')
mask_6[x,y] = 1
mask_7 = np.zeros((2160,4320),'float32')
mask_7[x,y] = 1

search_1 = np.zeros((2160,4320),'float32')
search_1[x,y] = 1
search_2 = np.zeros((2160,4320),'float32')
search_2[x,y] = 1
for a in range(x-1,x+2):
    for b in range(y-1,y+2):
        search_2[a,b] = 1
judgement = np.sum(search_2-search_1)
ran = 1

f_mask_6 = np.zeros((2160,4320),'float32')
f_mask_7 = np.zeros((2160,4320),'float32')

loop = 1
while judgement>0 and loop<loop_max:
    print(f"################## ran = {ran} ##################")
    for a in range(0,2160,1):
        for b in range(0,4320,1):
            if search_2[a,b]==1:
                if pop[a,b]>thre_6:
                    if mask_6[a-1,b-1]==1 or mask_6[a,b-1]==1 or mask_6[a+1,b-1]==1 or mask_6[a-1,b]==1 or mask_6[a+1,b]==1 or mask_6[a-1,b+1]==1 or mask_6[a,b+1]==1 or mask_6[a+1,b+1]==1:
                        mask_6[a,b] = 1
                        print(f"thre_6: a{a}, b {b}")
                elif pop[a,b]>thre_7:
                    if mask_7[a-1,b-1]==1 or mask_7[a,b-1]==1 or mask_7[a+1,b-1]==1 or mask_7[a-1,b]==1 or mask_7[a+1,b]==1 or mask_7[a-1,b+1]==1 or mask_7[a,b+1]==1 or mask_7[a+1,b+1]==1:
                        mask_7[a,b] = 1
                        print(f"thre_7: a{a}, b {b}")

    ran = ran + 1
    search_1 = np.copy(search_2)
    for c in range(x-ran,x+ran+1,1):
        for d in range(y-ran,y+ran+1,1):
            search_2[c,d] = 1
    search_2 = search_2-search_1

    judgement_6 = np.sum(mask_6-f_mask_6)
    f_mask_6 = np.copy(mask_6)
    judgement_7 = np.sum(mask_7-f_mask_7)
    f_mask_7 = np.copy(mask_7)
    JG = [judgement_6,judgement_7]
    judgement = max(JG)

    loop += 1
#-----------------------------------------------

mp_6 = mask_6*pop
pop_mask_6 = np.sum(mp_6)
mp_7 = mask_7*pop
pop_mask_7 = np.sum(mp_7)

pop_mask_list_3 = [pop_mask_6,pop_best_2,pop_mask_7]
print(f"pop_mask_list_3: {pop_mask_list_3}")
coverage_3 = [float(M/pop_city) for M in pop_mask_list_3]
print(f"coverage_3: {coverage_3}")
evl_3 = [abs(C-1.0) for C in coverage_3]
print(f"evl_3: {evl_3}")
min_evl_3 = min(evl_3)
best_index_3 = evl_3.index(min_evl_3)
best_coverage_3 = coverage_3[best_index_3]
pop_best_3 = pop_mask_list_3[best_index_3]
tt_best_3 = tt_list[best_index_3]
thre_best_3 = thre_list[best_index_3]

#-----------------------------------------------

if best_index_3==0:
    best_mask_3 = mask_6
elif best_index_3==1:
    best_mask_3 = best_mask_2
elif best_index_3==2:
    best_mask_3 = mask_7

grid_num = np.sum(best_mask_3)
city_A = best_mask_3*A
city_area = np.sum(city_A)

#------------------------------------------------
if tt_best_3 != 3750 and tt_best_3 != 250 and 0.8<=best_coverage_3<=1.2:
    # f"citynum {num}\n" \ #city number
    # f"threshold_1 {tt_best_3}\n" \ # population density threshold 
    # f"threshold_2 {thre_best_3}\n" \ # center population threshold
    # f"pop_mask {pop_best_3}\n" \ # estimated population
    # f"pop_city {pop_city}\n" \ # UN city list population
    # f"coverage {best_coverage_3}\n" \ # coverage rate
    # f"OK grid_num {grid_num}\n" \ # the number of city mask grids
    # f"city_area {city_area}\n" \ # city area
    # f"{city_name}\n") # city name
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"OK grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 != 3750 and tt_best_3 != 250:
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"NG grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 == 3750:
    print(f"citynum {num}\n" \
          f"threshold_1 {tt_best_3}\n" \
          f"threshold_2 {thre_best_3}\n" \
          f"pop_mask {pop_best_3}\n" \
          f"pop_city {pop_city}\n" \
          f"coverage {best_coverage_3}\n" \
          f"high grid_num {grid_num}\n" \
          f"city_area {city_area}\n" \
          f"{city_name}\n")
elif tt_best_3 == 250:
    if 0.8<=best_coverage_3<=1.2:
        print(f"citynum {num}\n" \
              f"threshold_1 {tt_best_3}\n" \
              f"threshold_2 {thre_best_3}\n" \
              f"pop_mask {pop_best_3}\n" \
              f"pop_city {pop_city}\n" \
              f"coverage {best_coverage_3}\n" \
              f"low_OK grid_num {grid_num}\n" \
              f"city_area {city_area}\n" \
              f"{city_name}\n")
    else:
        print(f"citynum {num}\n" \
              f"threshold_1 {tt_best_3}\n" \
              f"threshold_2 {thre_best_3}\n" \
              f"pop_mask {pop_best_3}\n" \
              f"pop_city {pop_city}\n" \
              f"coverage {best_coverage_3}\n" \
              f"low_NG grid_num {grid_num}\n" \
              f"city_area {city_area}\n" \
              f"{city_name}\n")

name = f'../../map/dat/cty_msk_/txt/city_{num}.txt'

################## ran = 1 ##################
thre_1: a1153, b 3441
thre_1: a1153, b 3442
thre_1: a1153, b 3443
thre_1: a1154, b 3441
thre_1: a1154, b 3442
thre_1: a1154, b 3443
thre_1: a1155, b 3441
thre_1: a1155, b 3442
thre_1: a1155, b 3443
################## ran = 2 ##################
thre_1: a1153, b 3440
thre_1: a1153, b 3444
thre_1: a1154, b 3440
thre_1: a1154, b 3444
thre_1: a1155, b 3440
thre_1: a1155, b 3444
thre_1: a1156, b 3440
thre_1: a1156, b 3441
thre_1: a1156, b 3442
thre_1: a1156, b 3443
thre_1: a1156, b 3444
pop_mask_list: [13469093.0, 1649652.5, 1649652.5]
coverage: [1.6053746126340882, 0.19662127532777116, 0.19662127532777116]
evl: [0.6053746126340882, 0.8033787246722288, 0.8033787246722288]
################## ran = 1 ##################
thre_4: a1153, b 3441
thre_4: a1153, b 3442
thre_4: a1153, b 3443
thre_4: a1154, b 3441
thre_4: a1154, b 3442
thre_4: a1154, b 3443
thre_4: a1155, b 3441
thre_4: a1155, b 3442
thre_4: a1155, b 3443
################## ran = 2 ########